In [1]:
import tensorflow as tf
import tf_encrypted as tfe

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'F:\Anaconda3\envs\tfe\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
#model class

class LogisticRegression:
  """Contains methods to build and train logistic regression."""
  def __init__(self, num_features):
    self.w = tfe.define_private_variable(
        tf.random_uniform([num_features, 1], -0.01, 0.01))
    self.w_masked = tfe.mask(self.w)
    self.b = tfe.define_private_variable(tf.zeros([1]))
    self.b_masked = tfe.mask(self.b)

  @property
  def weights(self):
    return self.w, self.b

  def forward(self, x):
    with tf.name_scope("forward"):
      out = tfe.matmul(x, self.w_masked) + self.b_masked
      y = tfe.sigmoid(out)
      return y

  def backward(self, x, dy, learning_rate=0.01):
    batch_size = x.shape.as_list()[0]
    with tf.name_scope("backward"):
      dw = tfe.matmul(tfe.transpose(x), dy) / batch_size
      db = tfe.reduce_sum(dy, axis=0) / batch_size
      assign_ops = [
          tfe.assign(self.w, self.w - dw * learning_rate),
          tfe.assign(self.b, self.b - db * learning_rate),
      ]
      return assign_ops

  def loss_grad(self, y, y_hat):
    with tf.name_scope("loss-grad"):
      dy = y_hat - y
      return dy

  def fit_batch(self, x, y):
    with tf.name_scope("fit-batch"):
      y_hat = self.forward(x)
      dy = self.loss_grad(y, y_hat)
      fit_batch_op = self.backward(x, dy)
      return fit_batch_op

  def fit(self, sess, x, y, num_batches):
    fit_batch_op = self.fit_batch(x, y)
    for batch in range(num_batches):
      print("Batch {0: >4d}".format(batch))
      sess.run(fit_batch_op, tag='fit-batch')

  def evaluate(self, sess, x, y, data_owner):
    """Return the accuracy"""
    def print_accuracy(y_hat, y) -> tf.Operation:
      with tf.name_scope("print-accuracy"):
        correct_prediction = tf.equal(tf.round(y_hat), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print_op = tf.print("Accuracy on {}:".format(data_owner.player_name),
                            accuracy)
        return print_op

    with tf.name_scope("evaluate"):
      y_hat = self.forward(x)
      print_accuracy_op = tfe.define_output(data_owner.player_name,
                                            [y_hat, y],
                                            print_accuracy)

    sess.run(print_accuracy_op, tag='evaluate')

In [3]:
# data owner class

class DataOwner:
  """Contains code meant to be executed by a data owner Player."""
  def __init__(
      self,
      player_name,
      num_features,
      training_set_size,
      test_set_size,
      batch_size
  ):
    self.player_name = player_name
    self.num_features = num_features
    self.training_set_size = training_set_size
    self.test_set_size = test_set_size
    self.batch_size = batch_size
    self.train_initializer = None
    self.test_initializer = None

  @property
  def initializer(self):
    return tf.group(self.train_initializer, self.test_initializer)

  @tfe.local_computation
  def provide_training_data(self):
    """Preprocess training dataset

    Return single batch of training dataset
    """
    def norm(x, y):
      return tf.cast(x, tf.float32), tf.expand_dims(y, 0)

    x_raw = tf.random.uniform(
        minval=-.5,
        maxval=.5,
        shape=[self.training_set_size, self.num_features])

    y_raw = tf.cast(tf.reduce_mean(x_raw, axis=1) > 0, dtype=tf.float32)

    train_set = tf.data.Dataset.from_tensor_slices((x_raw, y_raw)) \
        .map(norm) \
        .repeat() \
        .shuffle(buffer_size=self.batch_size) \
        .batch(self.batch_size)

    train_set_iterator = train_set.make_initializable_iterator()
    self.train_initializer = train_set_iterator.initializer

    x, y = train_set_iterator.get_next()
    x = tf.reshape(x, [self.batch_size, self.num_features])
    y = tf.reshape(y, [self.batch_size, 1])

    return x, y

  @tfe.local_computation
  def provide_testing_data(self):
    """Preprocess testing dataset

    Return single batch of testing dataset
    """
    def norm(x, y):
      return tf.cast(x, tf.float32), tf.expand_dims(y, 0)

    x_raw = tf.random.uniform(
        minval=-.5,
        maxval=.5,
        shape=[self.test_set_size, self.num_features])

    y_raw = tf.cast(tf.reduce_mean(x_raw, axis=1) > 0, dtype=tf.float32)

    test_set = tf.data.Dataset.from_tensor_slices((x_raw, y_raw)) \
        .map(norm) \
        .batch(self.test_set_size)

    test_set_iterator = test_set.make_initializable_iterator()
    self.test_initializer = test_set_iterator.initializer

    x, y = test_set_iterator.get_next()
    x = tf.reshape(x, [self.test_set_size, self.num_features])
    y = tf.reshape(y, [self.test_set_size, 1])

    return x, y

In [4]:
## class for model owner

class ModelOwner:
  """Contains code meant to be executed by a model owner Player."""
  def __init__(self, player_name):
    self.player_name = player_name

  @tfe.local_computation
  def receive_weights(self, *weights):
    return tf.print("Weights on {}:".format(self.player_name), weights)

In [5]:
## prediction client

class PredictionClient:
  """Contains methods meant to be executed by a prediction client."""
  def __init__(self, player_name, num_features):
    self.player_name = player_name
    self.num_features = num_features

  @tfe.local_computation
  def provide_input(self):
    return tf.random.uniform(
        minval=-.5,
        maxval=.5,
        dtype=tf.float32,
        shape=[1, self.num_features])

  @tfe.local_computation
  def receive_output(self, result):
    return tf.print("Result on {}:".format(self.player_name), result)

In [6]:
"""Dummy data preparation for logistic regression training."""
from typing import Tuple

import numpy as np
import tensorflow as tf

np.random.seed(1)


def norm(x: tf.Tensor, y: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
  return tf.cast(x, tf.float32), tf.expand_dims(y, 0)


def gen_training_input(total_size, nb_feats, batch_size):
  """Generate random data for training."""
  x_np = np.random.uniform(-.5, .5, size=[total_size, nb_feats])
  y_np = np.array(x_np.mean(axis=1) > 0, np.float32)
  train_set = tf.data.Dataset.from_tensor_slices((x_np, y_np)) \
                             .map(norm) \
                             .shuffle(buffer_size=100) \
                             .repeat() \
                             .batch(batch_size)
  train_set_iterator = train_set.make_one_shot_iterator()
  x, y = train_set_iterator.get_next()
  x = tf.reshape(x, [batch_size, nb_feats])
  y = tf.reshape(y, [batch_size, 1])

  # tf.print(x, data=[x], message="x: ", summarize=6)
  return x, y


def gen_test_input(total_size, nb_feats, batch_size):
  """Generate random data for evaluation."""
  x_test_np = np.random.uniform(-.5, .5, size=[total_size, nb_feats])
  y_test_np = np.array(x_test_np.mean(axis=1) > 0, np.float32)
  test_set = tf.data.Dataset.from_tensor_slices((x_test_np, y_test_np)) \
                            .map(norm) \
                            .batch(batch_size)
  test_set_iterator = test_set.make_one_shot_iterator()
  x_test, y_test = test_set_iterator.get_next()
  x_test = tf.reshape(x_test, [batch_size, nb_feats])
  y_test = tf.reshape(y_test, [batch_size, 1])

  return x_test, y_test


In [7]:
num_features = 10
training_set_size = 2000
test_set_size = 100
batch_size = 100
num_batches = (training_set_size // batch_size) * 10

model_owner = ModelOwner('model-owner')
data_owner_0 = DataOwner('data-owner-0',
                         num_features,
                         training_set_size,
                         test_set_size,
                         batch_size // 2)
data_owner_1 = DataOwner('data-owner-1',
                         num_features,
                         training_set_size,
                         test_set_size,
                         batch_size // 2)

tfe.set_protocol(tfe.protocol.Pond(
    tfe.get_config().get_player(data_owner_0.player_name),
    tfe.get_config().get_player(data_owner_1.player_name)
))

x_train_0, y_train_0 = data_owner_0.provide_training_data()
x_train_1, y_train_1 = data_owner_1.provide_training_data()

x_test_0, y_test_0 = data_owner_0.provide_testing_data()
x_test_1, y_test_1 = data_owner_1.provide_testing_data()

x_train = tfe.concat([x_train_0, x_train_1], axis=0)
y_train = tfe.concat([y_train_0, y_train_1], axis=0)

model = LogisticRegression(num_features)
reveal_weights_op = model_owner.receive_weights(model.weights)

with tfe.Session() as sess:
  sess.run([tfe.global_variables_initializer(),
            data_owner_0.initializer,
            data_owner_1.initializer],
           tag='init')

  model.fit(sess, x_train, y_train, num_batches)
  # TODO(Morten)
  # each evaluation results in nodes for a forward pass being added to the graph;
  # maybe there's some way to avoid this, even if it means only if the shapes match
  model.evaluate(sess, x_test_0, y_test_0, data_owner_0)
  model.evaluate(sess, x_test_1, y_test_1, data_owner_1)

  sess.run(reveal_weights_op, tag='reveal')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.




Batch    0
Batch    1
Batch    2
Batch    3
Batch    4
Batch    5
Batch    6
Batch    7
Batch    8
Batch    9
Batch   10
Batch   11
Batch   12
Batch   13
Batch   14
Batch   15
Batch   16
Batch   17
Batch   18
Batch   19
Batch   20
Batch   21
Batch   22
Batch   23
Batch   24
Batch   25
Batch   26
Batch   27
Batch   28
Batch   29
Batch   30
Batch   31
Batch   32
Batch   33
Ba

In [8]:
num_features = 10

model = LogisticRegression(num_features)
prediction_client_0 = PredictionClient('prediction-client-0', num_features // 2)
prediction_client_1 = PredictionClient('prediction-client-1', num_features // 2)
result_receiver = prediction_client_0

x_0 = prediction_client_0.provide_input()
x_1 = prediction_client_1.provide_input()
x = tfe.concat([x_0, x_1], axis=1)

y = model.forward(x)


reveal_output = result_receiver.receive_output(y)

with tfe.Session() as sess:
  sess.run(tfe.global_variables_initializer(), tag='init')

  sess.run(reveal_output, tag='predict')

Result on prediction-client-0: [[0.49794238683127573]]
